Corrigé de l'exo réservation

In [1]:
from pydantic import BaseModel, Field, computed_field

In [2]:
from datetime import date
from enum import Enum, auto
from uuid import UUID, uuid4

class Role(Enum):
    TEACHER = "TEACHER"
    STUDENT = "STUDENT"

class Person(BaseModel):
    pk: UUID = Field(default_factory=uuid4)
    first_name: str
    last_name: str
    date_of_birth: date
    role: Role

    @computed_field
    @property
    def email(self) -> str:
        return f"{self.first_name}.{self.last_name}@mines.org".lower()

    @computed_field
    @property
    def age(self) -> int:
        return int((date.today() - self.date_of_birth).days // 365.25)

    class Config:
        use_enum_values = True

In [3]:
Person(
    first_name="Jean",
    last_name="Dupont",
    date_of_birth=date(1972, 4, 7),
    role=Role.TEACHER,
)

Person(pk=UUID('64e7a4c3-92a5-431d-9756-8cd1e251d343'), first_name='Jean', last_name='Dupont', date_of_birth=datetime.date(1972, 4, 7), role='TEACHER', email='jean.dupont@mines.org', age=53)

In [4]:
Person(
    first_name="Jean",
    last_name="Dupont",
    date_of_birth="1972-06-12",
    role="TEACHER",
)

Person(pk=UUID('d5bc8013-43f2-4f1f-ac2e-ad6c1fe11eb1'), first_name='Jean', last_name='Dupont', date_of_birth=datetime.date(1972, 6, 12), role='TEACHER', email='jean.dupont@mines.org', age=53)

In [5]:
from functools import partial
import factory

Faker = partial(factory.Faker, locale="fr_FR")

class PersonFactory(factory.Factory):
    class Meta:
        model = Person

    first_name = Faker("first_name")
    last_name = Faker("last_name")
    date_of_birth = Faker(
        'date_between_dates',
        date_start=date(1960, 1, 1),
        date_end=date(2010, 12, 31),
    )
    role = factory.Iterator(Role)

In [6]:
PersonFactory.build()

Person(pk=UUID('93be7c42-841a-49ac-93f9-09cf6453559e'), first_name='Denise', last_name='Bigot', date_of_birth=datetime.date(2008, 5, 10), role='TEACHER', email='denise.bigot@mines.org', age=17)

In [7]:
import unittest
from pydantic import ValidationError

class TestPersonne(unittest.TestCase):
    def test_person_creation_nominal(self):
        person = Person(
            first_name="Jean",
            last_name="Dupont",
            date_of_birth=date(1972, 4, 7),
            role=Role.TEACHER,
        )
        self.assertEqual(person.email, "jean.dupont@mines.org")
        self.assertEqual(person.age, 53)

    def test_person_creation_coertion(self):
        person = Person(
            first_name="Jean",
            last_name="Dupont",
            date_of_birth="1972-04-07",
            role="TEACHER",
        )
        self.assertEqual(person.email, "jean.dupont@mines.org")
        self.assertEqual(person.age, 53)

    def test_person_creation_error_missing_first_name(self):
        with self.assertRaises(ValidationError):
            self.person = Person(
                last_name="Dupont",
                date_of_birth="1972-04-07",
                role="TEACHER",
            )

    def test_person_creation_error_missing_last_name(self):
        with self.assertRaises(ValidationError):
            self.person = Person(
                first_name="Jean",
                date_of_birth="1972-04-07",
                role="TEACHER",
            )

    def test_person_creation_error_missing_date_of_birth(self):
        with self.assertRaises(ValidationError):
            self.person = Person(
                first_name="Jean",
                last_name="Dupont",
                role="TEACHER",
            )

    def test_person_creation_error_missing_role(self):
        with self.assertRaises(ValidationError):
            self.person = Person(
                first_name="Jean",
                last_name="Dupont",
                date_of_birth="1972-04-07",
            )

    def test_person_creation_error_wrong_date_of_birth(self):
        with self.assertRaises(ValidationError):
            self.person = Person(
                first_name="Jean",
                last_name="Dupont",
                date_of_birth="1972-04_07",
                role="TEACHER",
            )

    def test_person_creation_error_wrong_role(self):
        with self.assertRaises(ValidationError):
            self.person = Person(
                first_name="Jean",
                last_name="Dupont",
                date_of_birth="1972-04-07",
                role="X",
            )


In [8]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_person_creation_coertion (__main__.TestPersonne.test_person_creation_coertion) ... ok
test_person_creation_error_missing_date_of_birth (__main__.TestPersonne.test_person_creation_error_missing_date_of_birth) ... ok
test_person_creation_error_missing_first_name (__main__.TestPersonne.test_person_creation_error_missing_first_name) ... ok
test_person_creation_error_missing_last_name (__main__.TestPersonne.test_person_creation_error_missing_last_name) ... ok
test_person_creation_error_missing_role (__main__.TestPersonne.test_person_creation_error_missing_role) ... ok
test_person_creation_error_wrong_date_of_birth (__main__.TestPersonne.test_person_creation_error_wrong_date_of_birth) ... ok
test_person_creation_error_wrong_role (__main__.TestPersonne.test_person_creation_error_wrong_role) ... ok
test_person_creation_nominal (__main__.TestPersonne.test_person_creation_nominal) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.004s

OK


In [9]:
persons = PersonFactory.build_batch(20)

In [10]:
import csv

fieldnames = ["pk", "first_name", "last_name", "email", "date_of_birth", "age", "role"]

with open("persons.csv", "w") as f:
    writer = csv.DictWriter(f, dialect=csv.unix_dialect, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows([person.model_dump() for person in persons])


In [11]:
with open("persons.csv") as f:
    print(f.read())

"pk","first_name","last_name","email","date_of_birth","age","role"
"53d89bbf-bf72-44b7-88ec-f801e8f0c761","André","Klein","andré.klein@mines.org","1967-08-31","57","STUDENT"
"9393d161-cd91-46f2-a99e-3f73e8e5fb74","Patricia","Ramos","patricia.ramos@mines.org","1995-10-27","29","TEACHER"
"f60ac261-41a5-425e-88f8-66cd2e19fec7","François","Maillot","françois.maillot@mines.org","1982-09-13","42","STUDENT"
"cacff65d-5fcf-48fe-811f-b9234c17595a","Matthieu","Dos Santos","matthieu.dos santos@mines.org","2010-07-06","14","TEACHER"
"147c6645-f5f7-466d-a130-0aea8fba36a2","Thibault","Dubois","thibault.dubois@mines.org","2004-04-21","21","STUDENT"
"b084c4c8-11d5-4a5e-9ad0-3e35b3cb045b","Capucine","Duhamel","capucine.duhamel@mines.org","2001-09-09","23","TEACHER"
"57f2e66b-81ba-4800-a4ff-110b3fb22663","Franck","Perret","franck.perret@mines.org","1985-08-17","39","STUDENT"
"af253171-acca-4e75-8941-392910238061","Timothée","Barbe","timothée.barbe@mines.org","1980-04-12","45","TEACHER"
"7dedacd3-7209-4c

In [12]:
with open("persons.csv") as f:
    reader = csv.DictReader(f, dialect=csv.unix_dialect)
    data = list(reader)

data = [Person.model_validate(datum) for datum in data]

In [13]:
data

[Person(pk=UUID('53d89bbf-bf72-44b7-88ec-f801e8f0c761'), first_name='André', last_name='Klein', date_of_birth=datetime.date(1967, 8, 31), role='STUDENT', email='andré.klein@mines.org', age=57),
 Person(pk=UUID('9393d161-cd91-46f2-a99e-3f73e8e5fb74'), first_name='Patricia', last_name='Ramos', date_of_birth=datetime.date(1995, 10, 27), role='TEACHER', email='patricia.ramos@mines.org', age=29),
 Person(pk=UUID('f60ac261-41a5-425e-88f8-66cd2e19fec7'), first_name='François', last_name='Maillot', date_of_birth=datetime.date(1982, 9, 13), role='STUDENT', email='françois.maillot@mines.org', age=42),
 Person(pk=UUID('cacff65d-5fcf-48fe-811f-b9234c17595a'), first_name='Matthieu', last_name='Dos Santos', date_of_birth=datetime.date(2010, 7, 6), role='TEACHER', email='matthieu.dos santos@mines.org', age=14),
 Person(pk=UUID('147c6645-f5f7-466d-a130-0aea8fba36a2'), first_name='Thibault', last_name='Dubois', date_of_birth=datetime.date(2004, 4, 21), role='STUDENT', email='thibault.dubois@mines.org',

---

In [14]:
from datetime import datetime, timedelta

class BookingStatus(Enum):
    DRAFT = "?"
    CONFIRMED = "OK"
    CANCELED = "KO"

class Room(BaseModel):
    number: int
    seating_capacity: int

class Booking(BaseModel):
    person: Person
    room: Room
    date_start: datetime
    date_end: datetime
    status: BookingStatus = Field(default=BookingStatus.DRAFT)

    @computed_field
    @property
    def duration(self) -> int:
        return (self.date_end - self.date_start).seconds

    @duration.setter
    def duration(self, duration: int):
        self.date_end = self.date_start + timedelta(seconds=duration)


In [15]:
class RoomFactory(factory.Factory):
    class Meta:
        model = Room

    number = factory.Sequence(lambda x: 100 + x)
    seating_capacity = Faker("pyint", min_value=44, max_value=88)

class BookingFactory(factory.Factory):
    class Meta:
        model = Booking

    person = factory.SubFactory(PersonFactory)
    room = factory.SubFactory(RoomFactory)
    date_start = Faker(
        'date_time_between_dates',
        datetime_start=datetime(1960, 1, 1),
        datetime_end=datetime(2010, 12, 31),
    )
    date_end = Faker(
        'date_time_between_dates',
        datetime_start=datetime(1960, 1, 1),
        datetime_end=datetime(2010, 12, 31),
    )
    status = factory.Iterator(BookingStatus)

In [16]:
person = PersonFactory.build()
room = Room(number=431, seating_capacity=50)

In [17]:
Booking(
    person=person,
    room=room,
    date_start=datetime(2025, 6, 13, 8),
    date_end=datetime(2025, 6, 13, 12),
)

Booking(person=Person(pk=UUID('da023200-f158-4dce-8fae-cbfedb0c3887'), first_name='Arthur', last_name='Sauvage', date_of_birth=datetime.date(1974, 6, 11), role='STUDENT', email='arthur.sauvage@mines.org', age=51), room=Room(number=431, seating_capacity=50), date_start=datetime.datetime(2025, 6, 13, 8, 0), date_end=datetime.datetime(2025, 6, 13, 12, 0), status=<BookingStatus.DRAFT: '?'>, duration=14400)

In [18]:
Booking(
    person=person,
    room=room,
    date_start="2025-06-13T08:00:00.000000",
    date_end="2025-06-13T17:00:00.000000",
)

Booking(person=Person(pk=UUID('da023200-f158-4dce-8fae-cbfedb0c3887'), first_name='Arthur', last_name='Sauvage', date_of_birth=datetime.date(1974, 6, 11), role='STUDENT', email='arthur.sauvage@mines.org', age=51), room=Room(number=431, seating_capacity=50), date_start=datetime.datetime(2025, 6, 13, 8, 0), date_end=datetime.datetime(2025, 6, 13, 17, 0), status=<BookingStatus.DRAFT: '?'>, duration=32400)

In [19]:
booking = Booking(
    person=person,
    room=room,
    date_start=datetime(2025, 6, 13, 8),
    date_end=datetime(2025, 6, 13, 12),
)
booking.duration = 32400
booking

Booking(person=Person(pk=UUID('da023200-f158-4dce-8fae-cbfedb0c3887'), first_name='Arthur', last_name='Sauvage', date_of_birth=datetime.date(1974, 6, 11), role='STUDENT', email='arthur.sauvage@mines.org', age=51), room=Room(number=431, seating_capacity=50), date_start=datetime.datetime(2025, 6, 13, 8, 0), date_end=datetime.datetime(2025, 6, 13, 17, 0), status=<BookingStatus.DRAFT: '?'>, duration=32400)

In [20]:
booking.model_dump_json()

'{"person":{"pk":"da023200-f158-4dce-8fae-cbfedb0c3887","first_name":"Arthur","last_name":"Sauvage","date_of_birth":"1974-06-11","role":"STUDENT","email":"arthur.sauvage@mines.org","age":51},"room":{"number":431,"seating_capacity":50},"date_start":"2025-06-13T08:00:00","date_end":"2025-06-13T17:00:00","status":"?","duration":32400}'

---

In [21]:
import uuid

from sqlalchemy.orm import declarative_base, Mapped, mapped_column
from sqlalchemy import Column, ForeignKey, Integer, String, Enum, UUID, Date, DateTime
from sqlalchemy.orm import relationship


Base = declarative_base()


class PersonSchema(Base):

    def __init__(self, **entries):
        self.__dict__.update(entries)

    __tablename__ = 'person'

    pk: Mapped[uuid.UUID] = mapped_column(UUID(as_uuid=True), primary_key=True)

    first_name: Mapped[str] = mapped_column(String)
    last_name: Mapped[str] = mapped_column(String)

    date_of_birth: Mapped[date] = mapped_column(Date)

    role: Mapped[Role] = mapped_column(Enum(Role))

    def __repr__(self):
        return f"<Person {self.first_name} {self.last_name}>"

class RoomSchema(Base):

    def __init__(self, **entries):
        self.__dict__.update(entries)

    __tablename__ = 'room'

    number: Mapped[int] = mapped_column(Integer, primary_key=True)
    seating_capacity: Mapped[int] = mapped_column(Integer)

class BookingSchema(Base):

    def __init__(self, **entries):
        self.__dict__.update(entries)

    __tablename__ = 'booking'

    pk: Mapped[uuid.UUID] = mapped_column(UUID(as_uuid=True), primary_key=True)

    person_id: Mapped[UUID] = mapped_column(ForeignKey("person.pk"))
    room_id: Mapped[int] = mapped_column(ForeignKey("room.number"))

    date_start: Mapped[datetime] = mapped_column(DateTime(timezone=True))
    date_end: Mapped[datetime] = mapped_column(DateTime(timezone=True))

    status: Mapped[BookingStatus] = mapped_column(Enum(BookingStatus))

    person = relationship(PersonSchema, backref="bookings")
    room = relationship(RoomSchema, backref="bookings")

from sqlalchemy import create_engine

engine = create_engine('sqlite:///booking.db')

Base.metadata.create_all(bind=engine)

In [34]:
from sqlalchemy.orm import sessionmaker

DBSession = sessionmaker(bind=engine)
session = DBSession()

In [23]:
person

Person(pk=UUID('da023200-f158-4dce-8fae-cbfedb0c3887'), first_name='Arthur', last_name='Sauvage', date_of_birth=datetime.date(1974, 6, 11), role='STUDENT', email='arthur.sauvage@mines.org', age=51)

In [24]:
new_person = PersonSchema(**person.model_dump())
session.add(new_person)
session.commit()

In [25]:
p = session.query(PersonSchema).first()

In [26]:
Person.model_validate(p.__dict__)

Person(pk=UUID('da023200-f158-4dce-8fae-cbfedb0c3887'), first_name='Arthur', last_name='Sauvage', date_of_birth=datetime.date(1974, 6, 11), role='STUDENT', email='arthur.sauvage@mines.org', age=51)

In [30]:
batch = BookingFactory.create_batch(20)

In [35]:
for booking in batch:
    person = PersonSchema(**booking.person.model_dump())
    session.add(person)
    room = RoomSchema(**booking.room.model_dump())
    session.add(room)
    session.flush()
    session.add(
        BookingSchema(
            pk=uuid.uuid4(),
            person_id=person.pk,
            room_id=room.number,
            date_start=booking.date_start,
            date_end=booking.date_end,
            status=booking.status,
        )
    )
session.commit()